In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('encoded_snp.csv', sep=' ')
snp = data.columns.values
X = data[snp].values
def convertLabel(data):
    label_maker = LabelEncoder()
    data['Encode'] = label_maker.fit_transform(data['Class'])
convertLabel(data)
y = data['Encode'].values

## Feature Selection

import sklearn.feature_selection as fs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.ensemble import RandomForestClassifier

# Mutual information filter selection
mi = mutual_info_classif(X, y)
sel = SelectPercentile(mutual_info_classif, percentile=50).fit(X, y)
MI_column = X.columns[sel.get_support()]

# Random Forest-Recursive Feature Elimination wrapper selection
rf = RandomForestClassifier(max_depth=3, n_estimators=200, random_state=42)
rfe = fs.RFE(rf, n_features_to_select=350, step=1000)
X = rfe.fit_transform(X, y)

# Identify selected SNPs
ranking = dict(zip(MI_column, rfe.estimator_.feature_importances_))
ranking = dict(itertools.islice(ranking.items(), n_features))
top_snp = list(ranking.keys())
top_snp = np.asarray(top_snp)
selected_snp = pd.DataFrame(top_snp)
outfile = 'selected_snp.csv'
selected_snp.to_csv(outfile, sep=' ', index=False)


## Classification

from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

counter = 1
y_pred = []
y_true = []

loo = LeaveOneOut()

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = SVC()
    clf = SVC(kernel='linear', C=1.0, probability=True)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    y_pred.append(pred)
    y_true.append(y_test)
    
    counter = counter + 1

y_pred = np.asarray(y_pred)
y_true = np.asarray(y_true)


## Results

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
auc = metrics.auc(fpr, tpr)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
print('AUC', auc)
print('Accuracy', accuracy)
print('Precision', precision)
print('Recall', recall)
print('Confusion matrix')
print(pd.DataFrame(confusion_matrix(y_true, y_pred, columns=['pred_pos', 'pred_neg'], index=['pos','neg'])))